## yolov8-detection

In [ ]:
# Create yolov8-detection label
import numpy as np
import pandas as pd
import os
import shutil

# Open csv file
train = pd.read_csv('/kaggle/input/hackathon-online-phuket-landmark-recognition/train.csv')

# Create folder
os.mkdir('/kaggle/working/labels')

# Create label's text file
for i in range(len(train)):
    Classes = train['label'][i]
    text = str(Classes) + " 0.5 0.5 1 1"
    L = len(train['id'][i])-4
    name = str(train['id'][i][0:L])+ ".txt"
    file = open('/kaggle/working/labels/' + name,'w')
    file.write(text)
    file.close()

# zip file and upload
shutil.make_archive('labels', 'zip', '/kaggle/working/labels/')

# Train yolov8-detection
!unzip -qq /content/drive/MyDrive/landmark/yolov8.zip -d /content/datasets
!git clone https://github.com/ultralytics/ultralytics
%pip -qq install ultralytics
from ultralytics import YOLO
model = YOLO("yolov8n.pt")
results = model.train(data="/content/ultralytics/ultralytics/datasets/coco128.yaml", epochs=100,imgsz=640)

## yolov8-cls

In [ ]:
# Create yolov8-cls label
import pandas as pd
import cv2
import os
import random

# Create train and cal folder
name_list = os.listdir('train_det\labels')
random.shuffle(name_list)
Count = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
for i in range(len(name_list)):
    f = open('train_det/labels/' + str(name_list[i]), "r")
    for x in f:
        Class = int(x.split()[0])
    L = len(name_list[i]) - 4
    name = str(name_list[i][0:L]) + ".jpg"
    if Count[Class] <= 40:
        img = cv2.imread('train_det/images/' + name)
        path = 'val/'+ str(Class) +'/'+name
        cv2.imwrite(path, img)
        path = 'train/' + str(Class) + '/' + name
        cv2.imwrite(path, img)
        Count[Class] = Count[Class] + 1
    else:
        img = cv2.imread('train_det/images/' + name)
        path = 'train/' + str(Class) + '/' + name
        cv2.imwrite(path, img)

# Train yolov8-cls
!unzip -qq /content/drive/MyDrive/landmark/yolov8.zip -d /content/datasets
!git clone https://github.com/ultralytics/ultralytics
%pip -qq install ultralytics
from ultralytics import YOLO
model = YOLO('yolov8x-cls.pt')  # load a pretrained model (recommended for training)
model.train(data='/content/datasets', epochs=160, imgsz=640)